<a href="https://colab.research.google.com/github/rcleveng/notebooks/blob/main/Ask_Questions_about_'The_Wealth_of_Nations'_using_RAG_with_Google_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple RAG langchain example using Google's Generate AI API.

You'll need an API key from Generative AI Studio.


In [ ]:
!pip install kaleido python-multipart tensorflow-probability fastapi uvicorn milvus[client] pymilvus langchain langchain-cli "langserve[all]" google.generativeai langchain-google-genai


Restart the kernel once dependencies are updated

In [4]:
quit()

Start Milvus vector database

In [2]:
import pyarrow
print(pyarrow.__version__)

import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
print("Using pymilvus version: ", pymilvus.__version__)

from milvus import default_server, debug_server

# Cleanup anything from the default server
if default_server.running:
  print("stopping the default server")
  default_server.stop()
print("Cleaning up old files from the default server")
default_server.cleanup()

# Cleanup anything from the debug server
if debug_server.running:
  print("stopping the debug server")
  debug_server.stop()
  connections.disconnect('default')
print("Cleaning up old files from the debug server")
debug_server.cleanup()

# Start your milvus debug server
print("Starting the debug server")
debug_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
try:
 connections.connect(host='127.0.0.1', port=debug_server.listen_port)
except Exception as e:
  print(e)
print("Connections: ", connections.list_connections())

# Check if the server is ready.
print("Started milvus server version ", utility.get_server_version())



14.0.2
Using pymilvus version:  2.3.5
Cleaning up old files from the default server
Cleaning up old files from the debug server
Starting the debug server


2024-01-20 23:02:46,199 [INFO] __init__.py:386: Milvus server is started


Connections:  [('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x7c6de32c4a00>)]
Started milvus server version  v2.3.5-lite


In [5]:
BOOK_URL="https://www.gutenberg.org/cache/epub/3300/pg3300.txt"



In [6]:

import requests
from google.colab import userdata

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatGooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Get the key from https://makersuite.google.com
# Add your PALM_KEY as a secret to colab.

google_api_key = userdata.get('PALM_KEY')

# Get text from URL
def get_data(url):
 r = requests.get(url)
 return r.text

BOOK_TEXT = get_data(BOOK_URL)


doc =  Document(page_content=BOOK_TEXT, metadata={"source": "local"})

text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents([doc])


# Count the number of chunks
print ('loaded chunks: ', len(texts))


loaded chunks:  3102


In [ ]:
# Create collection

from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI
from langchain.vectorstores import Milvus


embeddings = GooglePalmEmbeddings(google_api_key=google_api_key)
llm = GooglePalm(google_api_key=google_api_key)

# use
vector_db = Milvus.from_documents(
   texts,
   embeddings,
   connection_args={"host": "127.0.0.1", "port": default_server.listen_port},
)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_db.as_retriever())


In [17]:
# Ask question
query = """

How was the productive power of labor most increased?

"""

qa.run(query)

'the division of labor'

In [ ]:
# Stop your milvus server
default_server.stop()